In [2]:
import os
import utils
import random
import dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.eager as tfe

%matplotlib inline
#tfe.enable_eager_execution()
sess=tf.Session()

In [3]:
# load data
os.system("sh clean.sh")
X=dataset.load_dataset("./ShapeNet",render_count=10,num_of_examples=1)
Y=None;

[load_dataset] loading from ./ShapeNet
[construct_paths] parsing dir ./ShapeNet for ['.obj', '.mtl'] ...
... done
[write_renders_to_disk] writing renders to ./ShapeNet_Renders/02691156/b7b657d7db3c3b8cd13ef00338ba8c52 ... 
... done
[fetch_renders_from_disk] fetching renders from ./ShapeNet_Renders/02691156/b7b657d7db3c3b8cd13ef00338ba8c52 ... 
... done


In [4]:
# encoder network
with tf.name_scope("encoding_network"):
    cur_tensor=tf.convert_to_tensor(X[0],dtype=tf.float32)
    print(cur_tensor.shape)
    k_s = [3,3]
    conv_filter_count = [96, 128, 256, 256, 256, 256]
    for i in range(6): 
        ks=[7,7]if i is 0 else k_s  
        with tf.name_scope("encoding_block"):
            cur_tensor=tf.layers.conv2d(cur_tensor,filters=conv_filter_count[i],kernel_size= k_s,activation=None)
            cur_tensor=tf.layers.max_pooling2d(cur_tensor,2,2)
            cur_tensor=tf.nn.relu(cur_tensor)
            print(cur_tensor.shape)

    # flatten tensor
    cur_tensor=tf.contrib.layers.flatten(cur_tensor)
    print(cur_tensor.shape)
    cur_tensor=tf.contrib.layers.fully_connected(cur_tensor,1024)
    print(cur_tensor.shape)

(10, 512, 512, 3)
(10, 255, 255, 96)
(10, 126, 126, 128)
(10, 62, 62, 256)
(10, 30, 30, 256)
(10, 14, 14, 256)
(10, 6, 6, 256)
(10, 9216)
(10, 1024)


In [7]:
# 3D_Convolutional_LSTM 
print(cur_tensor.shape)
# create 4X4X4 grid of cells
cell_grid=utils.grid3D(tf.nn.rnn_cell.GRUCell(1024))
print(cell_grid.shape)

hidden_state_t=tf.zeros([3,3,3])
cur_tensor_t=cur_tensor
hidden_state_t=tf.layers.conv3d(cur_tensor,,[3,3,3])

for index, cell in np.ndenumerate(cell_grid):
    #cell_neighbors=utils.get_neighbors(cell_grid,loc=index)
    hidden_state_t=tf.nn.conv3d(cur_tensor,hidden_state_t,[1,1,1,1,1],'SAME')

    (cur_tensor_t,hidden_state_t)=cell(cur_tensor_t,hidden_state_t)
    
    
    
    

(10, 1, 1, 1, 1024)
(4, 4, 4)


ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv3d/Conv3D' (op: 'Conv3D') with input shapes: [10,1,1,1,1024], [3,3,3,1024,5].

In [ ]:
# decoding network
# batch_size=cur_tensor.shape.as_list()[0]
# cur_tensor=tf.reshape(cur_tensor,[batch_size,2,2,2,-1])
# print(cur_tensor.shape)
with tf.name_scope("decoding_network"):
    k_s = [3,3,3]
    deconv_filter_count = [128, 128, 128, 64, 32, 2]
    for i in range(6): 
        with tf.name_scope("decoding_block"):
                cur_tensor=tf.layers.conv3d_transpose(cur_tensor,padding='VALID',filters=deconv_filter_count[i],kernel_size= k_s,activation=None)
                cur_tensor=utils.unpool(cur_tensor)
                cur_tensor=tf.nn.relu(cur_tensor)
                print(cur_tensor.shape)

In [ ]:
# serialize graph for tensorboard
os.system("rm -rf ./logs/*")
writer = tf.summary.FileWriter("./logs/")
writer.add_graph(sess.graph)